In [6]:
# urn and other imports
import unityneuro.render as urn
import numpy as np
import pandas as pd
# ibl imports
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
from brainbox.io.one import SpikeSortingLoader
import ibllib.atlas as atlas
ba = atlas.AllenAtlas(25)

In [34]:
def get_bwm_sessions():

    '''
    Gaelle, 06.04.2022
    '''

    str_query = (  
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt'
       ',50,~json__qc,CRITICAL,'
       'session__extended_qc__behavior,1,'
       'json__extended_qc__tracing_exists,True,'
       '~session__extended_qc___task_stimOn_goCue_delays__lt,0.9,'
       '~session__extended_qc___task_response_feedback_delays__lt,0.9,'
       '~session__extended_qc___task_wheel_move_before_feedback__lt,0.9,'
       '~session__extended_qc___task_wheel_freeze_during_quiescence__lt,0.9,'
       '~session__extended_qc___task_error_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_correct_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_reward_volumes__lt,0.9,'
       '~session__extended_qc___task_reward_volume_set__lt,0.9,'
       '~session__extended_qc___task_stimulus_move_before_goCue__lt,0.9,'
       '~session__extended_qc___task_audio_pre_trial__lt,0.9')                  
        
    str_query2 = (
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt,50,'
       '~json__qc,CRITICAL,session__extended_qc__behavior,1,'       
       'json__extended_qc__tracing_exists,True,'
       'session__extended_qc___experimenter_task,PASS')        
            
    ins = np.concatenate([one.alyx.rest('insertions', 'list', django = x)
                          for x in [str_query, str_query2]])

    eid_probe = set([x['session']+'_'+x['name'] for x in ins]) # pid via x['id']
    ins = [x.split('_') for x in eid_probe] 
 
    return ins

In [35]:
ins = get_bwm_sessions()

In [36]:
ins[0]

['57b5ae8f-d446-4161-b439-b191c5e3e77b', 'probe01']

# Planned insertions

In [25]:
def getCoords(ins):
    entry_coords = ba.xyz2ccf(ins.entry)
    tip_coords = ba.xyz2ccf(ins.tip)
    tip_angles = [ins.phi, ins.theta, ins.beta]
    recording_size = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
    return (tip_coords, tip_angles, recording_size)

In [44]:
bwm_trajs = one.alyx.rest('trajectories', 'list', provenance='planned',
                                   project='ibl_neuropixel_brainwide_01', use_cache=False)


In [53]:
traj_data = pd.DataFrame(columns=['pid','lab','ml','ap','dv','phi','theta','depth'])

for i, bwmtraj in enumerate(bwm_trajs):
    traj =  one.alyx.rest('trajectories', 'list', provenance='planned',probe_insertion=bwmtraj['probe_insertion'],
                          project='ibl_neuropixel_brainwide_01', use_cache=False)
    traj = traj[0]
    ins = atlas.Insertion.from_dict(traj)

    (coords, angles, size) = getCoords(ins)
    
    traj_data.loc[i] = [traj['probe_insertion'],traj['session']['lab'],coords[0],coords[1],coords[2],angles[0],angles[1],0]
traj_data.to_csv('bwm_planned_traj.csv',float_format='%.03f')


In [54]:
traj_data

,pid,lab,ml,ap,dv,phi,theta,depth
0,98d9b1db-e58b-4780-9655-29768e5af716,churchlandlab,5031.276180,3900.0,5174.703305,180.0,15.0,0
1,232ce83a-4896-4a90-bfb7-1f2c51c2bee5,hausserlab,1423.687197,7550.0,5594.278676,0.0,20.0,0
2,19c5b0d5-a255-47ff-9f8d-639e634a7b61,cortexlab,5852.920386,3200.0,2695.796439,180.0,19.0,0
3,ef03e569-2b50-4534-89f4-fb1e56e81461,wittenlab,4531.276180,7400.0,4364.703305,180.0,15.0,0
4,78cb9d69-a49a-480f-b182-7f7985a5b1dd,churchlandlab,6946.723820,7400.0,4364.703305,0.0,15.0,0
...,...,...,...,...,...,...,...,...
932,94283796-548a-4e4e-909a-bd4bd1690d27,wittenlab,5984.723820,2900.0,5364.703305,0.0,15.0,0
933,ee320f72-b613-4acd-b10d-3ceef332a949,mrsicflogellab,2809.789514,8650.0,6689.344236,0.0,10.0,0
934,ba291bec-4492-4d7f-a6aa-483ebb64b3c3,danlab,5974.276180,8900.0,4151.703305,180.0,15.0,0
935,c2363000-27a6-461e-940b-15f681496ed8,danlab,4070.276180,6900.0,4555.703305,180.0,15.0,0


In [11]:
import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

In [22]:
colors = {}
labs = np.unique(traj_data['lab'].values)

for lab in labs:
    colors[lab] = randHexColor()

In [9]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [57]:
# VIEW 1: Planned insertions
urn.clear()
count = 0

for i, row in traj_data.iterrows():
    probename = 'p'+str(i)
    urn.create_probes([probename])
    urn.set_probe_positions({probename:[row.ml, row.ap, row.dv]})
    urn.set_probe_colors({probename:'#808080'})
    urn.set_probe_angles({probename:[row.phi, row.theta, 0]})
    urn.set_probe_size({probename:[0.07,3.84,0.02]})

# # for some reason 'root' doesn't work?
urn.set_area_visibility({8:True})
urn.set_area_material({8:'transparent-unlit'})
urn.set_area_color({8:'#000000'})
urn.set_area_alpha({8:0.025})

In [56]:
urn.clear()